In [2]:
from uplink import Consumer, get, post, Body, returns, Field, headers
from uplink import json as uplink_json
import xmltodict
import json
import time
import os
import re
import unicodedata
import codecs


Authorization

In [5]:
class UPAuth(Consumer):

    @returns.json
    @post("api/external/auth/token")
    def get_jwt_token(self, access_token: Field): pass


def get_auth_headers(token):
    return {
        "Authorization": "Bearer " +api_token
    }


swapi = UPAuth(base_url = "https://api.uprp.gov.pl/cmuia/")

In [6]:
# Obtain authorization JWT token
response = swapi.get_jwt_token(open("auth/.up_token").read())
api_token = response['token']

headers_search = get_auth_headers(api_token)

Search

In [7]:
class UPSearch(Consumer):
    
    @uplink_json
    @headers(headers_search)
    @post("api/external/deterministic-search")
    def register_query(self, query: Body): pass
    
    
    @headers(headers_search)
    @get("api/external/search-results/{query_id}?limit=10000")
    def search(self, query_id:str): pass
    

# Search 
up_search = UPSearch(base_url = "https://api.uprp.gov.pl/ewyszukiwarka/")

In [8]:
# Search query 
def search_by_submitter(submitter_name):
  return  {
    "includeDiacritical": "true",
    "attributes": [
      {"attributeId":144,"formType":"string","attributeName":"zglaszajacy.zglaszajacy","order":0,"logicalOperator":None,"stringOperator":"contains","text": submitter_name,"checked":None}
      #,{"attributeId":156,"formType":"date","attributeName":"data_zgloszenia","order":1,"logicalOperator":"and","stringOperator":"contains","text":None,"checked":None,"from":"1990-01-01 00:00:00","to":"2022-12-31 00:00:00"}
      #,{"attributeId":157,"formType":"date","attributeName":"data_zgloszenia_pct","order":2,"logicalOperator":"or","stringOperator":"contains","text":None,"checked":None,"from":"1990-01-01 00:00:00","to":"2022-12-31 00:00:00"}
    ],
    "pwpTypes": [
      #1,
      2,
      3,
      4,
      5,
      6
    ],
    "collections": [
      1,
      2,
      3,
      4,
      5,
      6,
      7,
      8
    ],
    "subCollections":["B1","B2","B3","B4","Y1","Y3","S2","T3","T4","T5","T6"]
  }
  

In [9]:
def query(submitter_name):

    # Substring of first 60 characters
    name_q = submitter_name[:60]
    
    # Register the query 
    response = up_search.register_query(search_by_submitter(name_q))#submitter_name))
    query_id = response.text

    # Execute the query
    response = up_search.search(query_id)
    #print(response.text)
   
    # Parse the response
    res = xmltodict.parse(response.text)
    return res

In [10]:
# Encode file name based on the string

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [11]:
import pandas as pd 
# Import zestawienie.xlsx file as pandas dataframe 
#df = pd.read_excel('zestawienie.xlsx')

institutions_df = pd.read_excel('out/polon_discp.xlsx')

In [12]:
# Jsonl writer
def write_jsonl(data, filename):
    with codecs.open(filename, 'a', encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item))
            f.write('\n')

In [14]:

# output dir
out_dir = "out/data/"

# Create dir structure if not exists
import os
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
    
#for inst in institutions:    
for inst_i in range(len(institutions_df)):
    
    id =  institutions_df.iloc[inst_i]['uid']
    
    inst = institutions_df.iloc[inst_i]['name']
    
    inst_name = institutions_df.iloc[inst_i]['name_clean']
    
    inst_fname = slugify(inst_name) + '.json'   
    
    inst_fname_loc = os.path.join(out_dir, inst_fname)

    # Check if file exists
    if not os.path.exists(inst_fname_loc):

        print("Processing: ", inst_name," file: ", inst_fname)
        res = query(inst_name)
        
        #print(res)
        data = {inst_fname_loc:{
            'id':           id,
            'name':         inst,
            'name_clean':   inst_name,
            'total':        res['root']['total'],
            'page':         res['root']['page'],
            'pages':        res['root']['pages'],
            'limit':        res['root']['limit']
        }}
            
        print("Processed: ", inst," file: ", inst_fname)
        json.dump(res, open(inst_fname_loc, 'w'), indent=4)
    
        write_jsonl([data], 'out/data.log')
        
        # Sleep for 1 second
        time.sleep(1)
    

Processing:  Centralny Ośrodek Badawczo-Rozwojowy Przemysłu Poligraficznego  file:  centralny-osrodek-badawczo-rozwojowy-przemysu-poligraficznego.json
Processed:  Centralny Ośrodek Badawczo-Rozwojowy Przemysłu Poligraficznego  file:  centralny-osrodek-badawczo-rozwojowy-przemysu-poligraficznego.json
Processing:  Ośrodek Badawczo-Rozwojowy Urządzeń Mechanicznych "OBRUM" sp. z o.o.  file:  osrodek-badawczo-rozwojowy-urzadzen-mechanicznych-obrum-sp-z-oo.json
Processed:  Ośrodek Badawczo-Rozwojowy Urządzeń Mechanicznych "OBRUM" sp. z o.o.  file:  osrodek-badawczo-rozwojowy-urzadzen-mechanicznych-obrum-sp-z-oo.json
Processing:  Centralne Laboratorium Ochrony Radiologicznej  file:  centralne-laboratorium-ochrony-radiologicznej.json
Processed:  Centralne Laboratorium Ochrony Radiologicznej  file:  centralne-laboratorium-ochrony-radiologicznej.json
Processing:  "Poltegor-Instytut" Instytut Górnictwa Odkrywkowego  file:  poltegor-instytut-instytut-gornictwa-odkrywkowego.json
Processed:  "Polteg